In [837]:
import psycopg2
import pandas as pd
import names
import uuid
import datetime

In [838]:
#%run functions.ipynb

In [839]:
database_setup = {
    'host' : '127.0.0.11'
    ,'database': 'ZSBD_p1'
    ,'user' : 'inzdanych123'
    ,'password' : 'inzdanych123'
    ,'port' : '5432'
}

In [840]:
database_setup

{'host': '127.0.0.11',
 'database': 'ZSBD_p1',
 'user': 'inzdanych123',
 'password': 'inzdanych123',
 'port': '5432'}

In [841]:
def do_query(query, print_out = False, commit = False):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()

        cur.execute(query)
        
        for notice in con.notices:
            print(notice)

        if commit:
            con.commit()

        if print_out:
            column_names = [desc[0] for desc in cur.description]
            row_list = []
            row = cur.fetchone()
            while row is not None:
                row_list.append(row)
                #print(row)
                row = cur.fetchone()
            df = pd.DataFrame(row_list, columns=column_names)
            print(df)
        
        cur.close()
        con.close()
        print('\n')
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# FUNCTIONS

In [842]:
def function_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [843]:
def function_tester(function, *arguments_of_it):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()
        
        if len(arguments_of_it) == 1:
            cur.execute(function + "('" +str(arguments_of_it[0]) + "')")
        else:
            cur.execute(function + str(arguments_of_it))

        column_names = [desc[0] for desc in cur.description]
        row_list = []
        row = cur.fetchone()
        while row is not None:
            row_list.append(row)
            #print(row)
            row = cur.fetchone()
        df = pd.DataFrame(row_list, columns=column_names)
        print(df)
        
        for notice in con.notices:
            print(notice)

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print("FUNTION TESTER ERROR")
            print(error)

    finally:
        if con is not None:
            con.close()

In [844]:
do_query("""
drop function if exists ai_med.ilosc_dni_lek_pacjent_(character, character, character);
""", 
commit=True)

function_def("""
create or replace function ai_med.ilosc_dni_lek_pacjent_(imieee char, nazwiskooo char, lek char)

returns table(
   id int,
   imie varchar,
   nazwisko varchar,
   data_rozpoczecie_brania_leku date,
   data_zakonczenia_brania_leku date,
   dni_brania_leku int
)

language plpgsql
as
$$

declare
   ilosc_pacjentow integer;

begin

   IF lek = 'ventolin'
      THEN
      return query
      SELECT 
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
      ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'atorvastatin' 
      THEN
      return query
      SELECT 
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol
      ,"Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'hydrochlorothiazide' 
      THEN
      return query
      SELECT
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi
      ,"Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   ELSE 
      RAISE EXCEPTION 'Nie ma takiego leku';
   END IF;

end;
$$;
""",
commit = True)

In [845]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "ventolin")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-05-07   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2020-08-04              820  


In [846]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "atorvastatin")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-02-16   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2020-09-14              941  


In [847]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "hydrochlorothiazide")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-12-30   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2021-02-27              790  


In [848]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheler", "jakislek")

FUNTION TESTER ERROR
Nie ma takiego leku
CONTEXT:  PL/pgSQL function ai_med.ilosc_dni_lek_pacjent_(character,character,character) line 54 at RAISE



In [849]:
do_query("""
drop function if exists ai_med.insert_nowego_pacjenta(varchar(57), varchar(57), date);
""", 
commit=True)

function_def("""
create or replace function ai_med.insert_nowego_pacjenta(p_imie varchar(57), p_nazwisko varchar(57), p_data_urodzenia date)

returns table(
   imie varchar(57),
   nazwisko varchar(57),
   data_urodzenia date
   )

language plpgsql
as
$$

begin

   INSERT INTO ai_med.pacjent
      (imie, nazwisko, data_urodzenia)
   SELECT p_imie, p_nazwisko, p_data_urodzenia
   WHERE
      NOT EXISTS (
         SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
         WHERE pacjent.imie = p_imie AND pacjent.nazwisko = p_nazwisko AND pacjent.data_urodzenia = p_data_urodzenia
    );
    

   RAISE NOTICE 'Dane dla pacjenta % %, zostaly wprowadzone.', p_imie, p_nazwisko;
   
   return query
   SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
   WHERE pacjent.imie = p_imie AND pacjent.nazwisko = p_nazwisko AND pacjent.data_urodzenia = p_data_urodzenia;

end;
$$;
""",
commit = True)

In [850]:
imie = 'Jan'
nazwisko = 'Nowak'
data_urodzenia = '1998-11-22'

In [851]:
do_query(("""
SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
WHERE pacjent.imie = '%s' AND pacjent.nazwisko = '%s' AND pacjent.data_urodzenia = '%s'
""") % (imie, nazwisko, data_urodzenia),
commit=True,
print_out=True)

  imie nazwisko data_urodzenia
0  Jan    Nowak     1998-11-22




In [852]:
for i in range(0,3):
    do_query("SELECT * FROM ai_med.insert_nowego_pacjenta('%s','%s','%s')" % (imie, nazwisko, data_urodzenia),
    commit=True,
    print_out=True)

NOTICE:  Dane dla pacjenta Jan Nowak, zostaly wprowadzone.

  imie nazwisko data_urodzenia
0  Jan    Nowak     1998-11-22


NOTICE:  Dane dla pacjenta Jan Nowak, zostaly wprowadzone.

  imie nazwisko data_urodzenia
0  Jan    Nowak     1998-11-22


NOTICE:  Dane dla pacjenta Jan Nowak, zostaly wprowadzone.

  imie nazwisko data_urodzenia
0  Jan    Nowak     1998-11-22




In [853]:
do_query(("""
SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
WHERE pacjent.imie = '%s' AND pacjent.nazwisko = '%s' AND pacjent.data_urodzenia = '%s'
""") % (imie, nazwisko, data_urodzenia),
commit=True,
print_out=True)

  imie nazwisko data_urodzenia
0  Jan    Nowak     1998-11-22




# TRIGGERS

In [854]:
def trigger_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [855]:
trigger_def("""
CREATE TRIGGER pacjent_insert_timestamp 
    AFTER
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.pacjent_insert_timestamp_func();
""",
commit=True)

trigger "pacjent_insert_timestamp" for relation "pacjent" already exists



In [856]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.pacjent_insert_timestamp_func() RETURNS TRIGGER AS $pacjent_table$
   BEGIN
      INSERT INTO ai_med.pacjent_insert_timestamp_table(id, ostatni_insert_danych) VALUES (NEW.id ,date_trunc('day', current_timestamp));
      RETURN NEW;
   END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

In [857]:
do_query("""
DROP TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
""", 
commit=True)
do_query("""
CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
(
    id serial NOT NULL,
    ostatni_insert_danych date NOT NULL,
    ostatnie_powiadomienie_dla_MLE date
);
""", 
commit=True)

In [858]:
do_query("""
ALTER TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
    ADD FOREIGN KEY (id)
    REFERENCES ai_med.pacjent (id) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;
""", 
commit=True)

In [859]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Dwight', 'Schrute', '1962-11-23');
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent where imie ='Dwight' and nazwisko = 'Schrute';
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

'NoneType' object is not iterable
       id    imie nazwisko data_urodzenia                       created_at
0  110853  Dwight  Schrute     1962-11-23 2022-11-24 17:06:58.924089+01:00
1  110854  Dwight  Schrute     1962-11-23 2022-11-24 17:21:18.163307+01:00
2  110855  Dwight  Schrute     1962-11-23 2022-11-24 17:21:22.065947+01:00
3  110892  Dwight  Schrute     1962-11-23 2022-11-25 19:22:01.986472+01:00
4  110953  Dwight  Schrute     1962-11-23 2022-11-26 19:53:33.357679+01:00
5  110957  Dwight  Schrute     1962-11-23 2022-11-26 19:58:57.996987+01:00


       id ostatni_insert_danych ostatnie_powiadomienie_dla_mle
0  110957            2022-11-26                           None




In [860]:
trigger_def("""
CREATE OR REPLACE TRIGGER message_to_ML_E
    BEFORE
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.message_to_ML_E_func();
""",
commit=True)

In [861]:
#TO W SUMIE POWINNO BYc NA KONIEC JAKIEJŚ TRANZAKCJI
#W SENSIE ŻEBY ZROBIĆ TRANZAKCJĘ, KTÓRA ZAJMIE SIĘ DAILY INSERTEM DO TABEL
function_def("""
CREATE OR REPLACE FUNCTION ai_med.message_to_ML_E_func() RETURNS TRIGGER AS $pacjent_table$
	declare
		liczba_wprowadzonych int;
	BEGIN
		SELECT count(*) into liczba_wprowadzonych from ai_med.pacjent_insert_timestamp_table 
		where ostatni_insert_danych = date_trunc('day', current_timestamp);
		
		IF liczba_wprowadzonych = 5
		THEN
		UPDATE ai_med.pacjent_insert_timestamp_table
		SET ostatnie_powiadomienie_dla_mle=date_trunc('minute', current_timestamp);
		raise notice 'Dzis zostalo wprowadzonych przynajmniej % nowych pacjentow, moze chcecie zrobic douczanie modelu?', 
		
		liczba_wprowadzonych + 1;
		
		END IF;
		RETURN NEW;
   	END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

#może jakoś że jak jest mniej insertów danego dnia niż x (5 powiedzmy) to rollback?

In [862]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2002-11-23');
""", 
commit=True,
print_out=True)

do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

'NoneType' object is not iterable
       id ostatni_insert_danych ostatnie_powiadomienie_dla_mle
0  110957            2022-11-26                           None
1  110958            2022-11-26                           None




# PROCEDURES

In [863]:
def procedure_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# 1. Inserting data using a procedure GENEROWANIE danych za pomocą procedur?

# WYMAGANIA DO PROJEKTU NOTATKA TWOJA I TE PUNKTY CO SĄ POD SPODEM

In [864]:
do_query("""
drop function if exists ai_med.id_pacjenta(varchar(57), varchar(57), date);
""", 
commit=True)

function_def("""
create or replace function ai_med.id_pacjenta(
   p_imie varchar(57), p_nazwisko varchar(57), p_data_urodzenia date
   , OUT pacjent_id int
   )

language plpgsql
as
$$

begin

   SELECT id into pacjent_id from ai_med.pacjent WHERE pacjent.imie = p_imie AND pacjent.nazwisko = p_nazwisko AND pacjent.data_urodzenia = p_data_urodzenia;

   RAISE NOTICE 'ID pacjenta to %.', pacjent_id;

end;
$$;
""",
commit = True)

In [865]:
do_query("SELECT * FROM ai_med.id_pacjenta('%s','%s','%s')" % (imie, nazwisko, data_urodzenia),
    commit=True,
    print_out=True)

NOTICE:  ID pacjenta to 110894.

   pacjent_id
0      110894




In [866]:
#########bez problemu z kluczem (sama tabela morfologia)

do_query("""
DROP PROCEDURE IF EXISTS ai_med.database_insert_procedure(varchar(57), varchar(57), date)
""", 
commit=True)

procedure_def(
"""
create or replace procedure ai_med.database_insert_procedure(
    
    --dane pacjent 
    p_imie varchar(57), p_nazwisko varchar(57), p_data_urodzenia date
    
    --dane leki
    ,data_r_atorvastatin_calcium date, data_r_ventolin date, data_r_hydrochlorothiazide date
    ,data_z_atorvastatin_calcium date, data_z_ventolin date, data_z_hydrochlorothiazide date

    --dane choroby
    ,data_r_leczenia_u07 date, data_r_leczenia_i10 date, data_r_leczenia_j45 date
    ,data_z_leczenia_u07 date, data_z_leczenia_i10 date, data_z_leczenia_j45 date
    
    ------------------
    --dane morfologia
    ,il_erytrocytow real, st_hemoglobiny real, V_mcv real, V_mch real, V_mchc real, V_rdw real 
    ,data_wyk_morfologii date

    --dane notatki lekarskie
    ,lek_rodzinny varchar(100), radiol varchar(100)

    --dane objawy
    ,bol_glow boolean, zmecz boolean, data_obs_objawow date

    --dane obrazy
    ,radio varchar(100), histop varchar(100)

    --dane operacje
    ,d_przeszczepu_serca date, d_wyciecia_migdalkow date

    )
language plpgsql    
as $$

declare
    id_pacjenta_do_innych_tabel int = 1;

begin
    --dane pacjent
    INSERT INTO ai_med.pacjent
      (imie, nazwisko, data_urodzenia)
    SELECT p_imie, p_nazwisko, p_data_urodzenia
    WHERE
      NOT EXISTS (
         SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
         WHERE pacjent.imie = p_imie AND pacjent.nazwisko = p_nazwisko AND pacjent.data_urodzenia = p_data_urodzenia
    );

    --SELECT * into id_pacjenta_do_innych_tabel FROM ai_med.id_pacjenta(p_imie, p_nazwisko, p_data_urodzenia);
    
    

    ----------------
    --dane morfologia
    INSERT INTO ai_med.morfologia(
    ilosc_erytrocytow, stezenie_hemoglobiny, mcv, mch, mchc, rdw 
    ,pacjentid
    ,data_wykonanej_morfologii
    )
    VALUES 
    (
    il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw 
    ,id_pacjenta_do_innych_tabel
    ,data_wyk_morfologii
    );

    
    

end;
$$
""",
commit=True)

NOTICE:  procedure ai_med.database_insert_procedure(pg_catalog.varchar,pg_catalog.varchar,date) does not exist, skipping





In [867]:
##############to sprawdz czy działa po ogarnięciu KLUCZY

do_query("""
DROP PROCEDURE IF EXISTS ai_med.database_insert_procedure(varchar(57), varchar(57), date)
""", 
commit=True)

procedure_def(
"""
create or replace procedure ai_med.database_insert_procedure(
    
    --dane pacjent 
    p_imie varchar(57), p_nazwisko varchar(57), p_data_urodzenia date
    
    --dane leki
    ,data_r_atorvastatin_calcium date, data_r_ventolin date, data_r_hydrochlorothiazide date
    ,data_z_atorvastatin_calcium date, data_z_ventolin date, data_z_hydrochlorothiazide date

    --dane choroby
    ,data_r_leczenia_u07 date, data_r_leczenia_i10 date, data_r_leczenia_j45 date
    ,data_z_leczenia_u07 date, data_z_leczenia_i10 date, data_z_leczenia_j45 date
    
    ------------------
    --dane morfologia
    ,il_erytrocytow real, st_hemoglobiny real, V_mcv real, V_mch real, V_mchc real, V_rdw real 
    ,data_wyk_morfologii date

    --dane notatki lekarskie
    ,lek_rodzinny varchar(100), radiol varchar(100)

    --dane objawy
    ,bol_glow boolean, zmecz boolean, data_obs_objawow date

    --dane obrazy
    ,radio varchar(100), histop varchar(100)

    --dane operacje
    ,d_przeszczepu_serca date, d_wyciecia_migdalkow date

    )
language plpgsql    
as $$

declare
    id_pacjenta_do_innych_tabel int = 1;

begin
    --dane pacjent
    INSERT INTO ai_med.pacjent
      (imie, nazwisko, data_urodzenia)
    SELECT p_imie, p_nazwisko, p_data_urodzenia
    WHERE
      NOT EXISTS (
         SELECT pacjent.imie, pacjent.nazwisko, pacjent.data_urodzenia FROM ai_med.pacjent 
         WHERE pacjent.imie = p_imie AND pacjent.nazwisko = p_nazwisko AND pacjent.data_urodzenia = p_data_urodzenia
    );

    --SELECT * into id_pacjenta_do_innych_tabel FROM ai_med.id_pacjenta(p_imie, p_nazwisko, p_data_urodzenia);
    
    --dane leki
    INSERT INTO ai_med."Aktualnie przyjmowane leki"(
    id
    ,data_rozpoczecia_atorvastatin_calcium__cholesterol, data_rozpoczecia_ventolin__asthma, data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi
    ,data_zakonczenia_atorvastatin_calcium__cholesterol, data_zakonczenia_ventolin__asthma, data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi)
    VALUES 
    (id_pacjenta_do_innych_tabel
    ,data_r_atorvastatin_calcium, data_r_ventolin, data_r_hydrochlorothiazide
    ,data_z_atorvastatin_calcium, data_z_ventolin, data_z_hydrochlorothiazide);
    
    --dane choroby
    INSERT INTO ai_med."leczenie_w_kierunku_kod_ICD_10"(
    id
    ,data_rozpoczecia_leczenia_u07, data_rozpoczecia_leczenia_i10, data_rozpoczecia_leczenia_j45
    ,data_zakonczenia_leczenia_u07, data_zakonczenia_leczenia_i10, data_zakonczenia_leczenia_j45
    )
    VALUES 
    (id_pacjenta_do_innych_tabel
    ,data_r_leczenia_u07, data_r_leczenia_i10, data_r_leczenia_j45
    ,data_z_leczenia_u07, data_z_leczenia_i10, data_z_leczenia_j45);

    ----------------
    --dane morfologia
    INSERT INTO ai_med.morfologia(
    ilosc_erytrocytow, stezenie_hemoglobiny, mcv, mch, mchc, rdw 
    ,pacjentid
    ,data_wykonanej_morfologii
    )
    VALUES 
    (
    il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw 
    ,id_pacjenta_do_innych_tabel
    ,data_wyk_morfologii
    );

    --dane notatki lekarskie
    INSERT INTO ai_med."Nazwa pliku z notatkami lekarskimi"(
    lekarz_rodzinny, radiolog
    , pacjentid
    )
    VALUES 
    (
    lek_rodzinny, radiol
    ,id_pacjenta_do_innych_tabel
    );

    --dane objawy
    INSERT INTO ai_med.Objawy(
    id
    ,bol_glowy, zmeczenie, data_obserwowanych_objawow
    )
    VALUES 
    (
    id_pacjenta_do_innych_tabel
    ,bol_glow, zmecz, data_obs_objawow
    );

    --dane obrazy
    INSERT INTO ai_med.Obrazy(
    radiologiczne, histopatologiczne
    ,pacjentid
    )
    VALUES 
    (
    radio, histop
    ,id_pacjenta_do_innych_tabel
    );

    --dane operacje
    INSERT INTO ai_med.Operacje(
    id
    ,data_przeszczepu_serca, data_wyciecia_migdalkow
    )
    VALUES 
    (
    id_pacjenta_do_innych_tabel
    ,d_przeszczepu_serca, d_wyciecia_migdalkow
    );
    

end;
$$
""",
commit=True)

NOTICE:  procedure ai_med.database_insert_procedure(pg_catalog.varchar,pg_catalog.varchar,date) does not exist, skipping





In [868]:
do_query("""
DROP PROCEDURE IF EXISTS ai_med.database_insert_procedure(varchar(57), varchar(57), date)
""", 
commit=True)

procedure_def(
"""
create or replace procedure ai_med.database_insert_procedure(
    
    --dane pacjent 
    p_imie varchar(57), p_nazwisko varchar(57), p_data_urodzenia date
    
    --dane leki
    ,data_r_atorvastatin_calcium date, data_r_ventolin date, data_r_hydrochlorothiazide date
    ,data_z_atorvastatin_calcium date, data_z_ventolin date, data_z_hydrochlorothiazide date

    --dane choroby
    ,data_r_leczenia_u07 date, data_r_leczenia_i10 date, data_r_leczenia_j45 date
    ,data_z_leczenia_u07 date, data_z_leczenia_i10 date, data_z_leczenia_j45 date
    
    ------------------
    --dane morfologia
    ,il_erytrocytow real, st_hemoglobiny real, V_mcv real, V_mch real, V_mchc real, V_rdw real 
    ,data_wyk_morfologii date

    --dane notatki lekarskie
    ,lek_rodzinny varchar(100), radiol varchar(100)

    --dane objawy
    ,bol_glow boolean, zmecz boolean, data_obs_objawow date

    --dane obrazy
    ,radio varchar(100), histop varchar(100)

    --dane operacje
    ,d_przeszczepu_serca date, d_wyciecia_migdalkow date

    )
language plpgsql    
as $$

declare
    id_pacjenta_do_innych_tabel int;

begin
    --dane pacjent
    PERFORM FROM ai_med.insert_nowego_pacjenta(p_imie, p_nazwisko, p_data_urodzenia);
    SELECT * into id_pacjenta_do_innych_tabel FROM ai_med.id_pacjenta(p_imie, p_nazwisko, p_data_urodzenia);
    
    --dane leki
    INSERT INTO ai_med."Aktualnie przyjmowane leki"(
    id
    ,data_rozpoczecia_atorvastatin_calcium__cholesterol, data_rozpoczecia_ventolin__asthma, data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi
    ,data_zakonczenia_atorvastatin_calcium__cholesterol, data_zakonczenia_ventolin__asthma, data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi)
    VALUES 
    (id_pacjenta_do_innych_tabel
    ,data_r_atorvastatin_calcium, data_r_ventolin, data_r_hydrochlorothiazide
    ,data_z_atorvastatin_calcium, data_z_ventolin, data_z_hydrochlorothiazide);
    
    --dane choroby
    INSERT INTO ai_med."leczenie_w_kierunku_kod_ICD_10"(
    id
    ,data_rozpoczecia_leczenia_u07, data_rozpoczecia_leczenia_i10, data_rozpoczecia_leczenia_j45
    ,data_zakonczenia_leczenia_u07, data_zakonczenia_leczenia_i10, data_zakonczenia_leczenia_j45
    )
    VALUES 
    (id_pacjenta_do_innych_tabel
    ,data_r_leczenia_u07, data_r_leczenia_i10, data_r_leczenia_j45
    ,data_z_leczenia_u07, data_z_leczenia_i10, data_z_leczenia_j45);

    ----------------
    --dane morfologia
    INSERT INTO ai_med.morfologia(
    ilosc_erytrocytow, stezenie_hemoglobiny, mcv, mch, mchc, rdw 
    ,pacjentid
    ,data_wykonanej_morfologii
    )
    VALUES 
    (
    il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw 
    ,id_pacjenta_do_innych_tabel
    ,data_wyk_morfologii
    );

    --dane notatki lekarskie
    INSERT INTO ai_med."Nazwa pliku z notatkami lekarskimi"(
    lekarz_rodzinny, radiolog
    , pacjentid
    )
    VALUES 
    (
    lek_rodzinny, radiol
    ,id_pacjenta_do_innych_tabel
    );

    --dane objawy
    INSERT INTO ai_med.Objawy(
    id
    ,bol_glowy, zmeczenie, data_obserwowanych_objawow
    )
    VALUES 
    (
    id_pacjenta_do_innych_tabel
    ,bol_glow, zmecz, data_obs_objawow
    );

    --dane obrazy
    INSERT INTO ai_med.Obrazy(
    radiologiczne, histopatologiczne
    ,pacjentid
    )
    VALUES 
    (
    radio, histop
    ,id_pacjenta_do_innych_tabel
    );

    --dane operacje
    INSERT INTO ai_med.Operacje(
    id
    ,data_przeszczepu_serca, data_wyciecia_migdalkow
    )
    VALUES 
    (
    id_pacjenta_do_innych_tabel
    ,d_przeszczepu_serca, d_wyciecia_migdalkow
    );
    

end;
$$
""",
commit=True)

NOTICE:  procedure ai_med.database_insert_procedure(pg_catalog.varchar,pg_catalog.varchar,date) does not exist, skipping





In [869]:
%run functions.ipynb

In [870]:
imie = names.get_first_name()
nazwisko = names.get_last_name()
data_urodzenia = random_date(datetime.date(1920, 1, 1), datetime.date(2004, 2, 1))
data_r_atorvastatin_calcium, data_r_ventolin, data_r_hydrochlorothiazide, data_z_atorvastatin_calcium, data_z_ventolin, data_z_hydrochlorothiazide = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
data_r_leczenia_u07, data_r_leczenia_i10, data_r_leczenia_j45, data_z_leczenia_u07, data_z_leczenia_i10, data_z_leczenia_j45 = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw, data_wyk_morfologii = random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
lek_rodzinny, radiol = 'lekarz_rodzinny_' + str(uuid.uuid1()), 'radiolog_' + str(uuid.uuid1())
bol_glow, zmecz, data_obs_objawow = bool(random.getrandbits(1)), bool(random.getrandbits(1)), random_date(datetime.date(2010, 2, 1), datetime.date(2022, 2, 1))
radio, histop = 'obraz_radiologiczny_' + str(uuid.uuid1()), 'obraz_histopatologiczny_' + str(uuid.uuid1())
d_przeszczepu_serca, d_wyciecia_migdalkow = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1))


# MOZNA DODAC ZEBY DLA TEGO SAME GO PACJENTA DAWAC 2/3 iles danych ekstra (bo jest to teraz - przez funkcje wewnatrze tej procedury ze pacjent jest unikalny)

In [871]:
ilosc = 2
for i in range(0, ilosc):
    imie = names.get_first_name()
    nazwisko = names.get_last_name()
    data_urodzenia = random_date(datetime.date(1920, 1, 1), datetime.date(2004, 2, 1))
    data_r_atorvastatin_calcium, data_r_ventolin, data_r_hydrochlorothiazide, data_z_atorvastatin_calcium, data_z_ventolin, data_z_hydrochlorothiazide = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
    data_r_leczenia_u07, data_r_leczenia_i10, data_r_leczenia_j45, data_z_leczenia_u07, data_z_leczenia_i10, data_z_leczenia_j45 = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1)), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
    il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw, data_wyk_morfologii = random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random.uniform(20.0, 60.6), random_date(datetime.date(2020, 2, 1), datetime.date(2022, 2, 1))
    lek_rodzinny, radiol = 'lekarz_rodzinny_' + str(uuid.uuid1()), 'radiolog_' + str(uuid.uuid1())
    bol_glow, zmecz, data_obs_objawow = bool(random.getrandbits(1)), bool(random.getrandbits(1)), random_date(datetime.date(2010, 2, 1), datetime.date(2022, 2, 1))
    radio, histop = 'obraz_radiologiczny_' + str(uuid.uuid1()), 'obraz_histopatologiczny_' + str(uuid.uuid1())
    d_przeszczepu_serca, d_wyciecia_migdalkow = random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1)), random_date(datetime.date(2018, 1, 1), datetime.date(2020, 2, 1))

    do_query("""call ai_med.database_insert_procedure(
            --dane pacjent
            '%s','%s','%s'

            --dane leki
            ,'%s', '%s', '%s', '%s', '%s', '%s'

            --dane choroby
            ,'%s', '%s', '%s', '%s', '%s', '%s'
            
            ---------------
            --dane morfologia
            ,%f, %f, %f, %f, %f, %f, '%s'
            --dane notatki lekarskie
            ,'%s', '%s'
            --dane objawy
            ,'%s', '%s', '%s'
            --dane obrazy
            ,'%s', '%s'
            --dane operacje 
            ,'%s', '%s'
            
        )""" % 
        (
            #dane pacjent
            imie, nazwisko, data_urodzenia

            #dane leki
            ,data_r_atorvastatin_calcium, data_r_ventolin, data_r_hydrochlorothiazide 
            ,data_z_atorvastatin_calcium, data_z_ventolin, data_z_hydrochlorothiazide

            # dane choroby
            ,data_r_leczenia_u07, data_r_leczenia_i10, data_r_leczenia_j45
            ,data_z_leczenia_u07, data_z_leczenia_i10, data_z_leczenia_j45
            
            #############
            # dane morfologia
            ,il_erytrocytow, st_hemoglobiny, V_mcv, V_mch, V_mchc, V_rdw, data_wyk_morfologii
            
            # dane notatki lekarskie
            ,lek_rodzinny, radiol

            # dane objawy
            ,bol_glow, zmecz, data_obs_objawow

            # dane obrazy
            ,radio, histop
            
            # dane operacje  
            ,d_przeszczepu_serca, d_wyciecia_migdalkow

        ),
        commit=True,
        print_out=True)

NOTICE:  Dane dla pacjenta Mark Rivera, zostaly wprowadzone.

NOTICE:  ID pacjenta to 110959.

'NoneType' object is not iterable
NOTICE:  Dane dla pacjenta Robert Hernandez, zostaly wprowadzone.

NOTICE:  ID pacjenta to 110960.

'NoneType' object is not iterable


In [872]:
do_query(
"""
SELECT * FROM ai_med.morfologia where pacjentid =1;  
""",
    print_out=True
)

      id  ilosc_erytrocytow  stezenie_hemoglobiny        mcv        mch  \
0  50030          50.327534             39.911705  59.013668  60.392310   
1  50031          26.661276             46.611324  36.906116  35.261890   
2  50032          50.817620             22.929173  50.425710  48.381664   
3  50033          54.572052             28.771875  24.529812  49.856440   

        mchc        rdw  pacjentid data_wykonanej_morfologii  
0  35.258484  26.867819          1                2020-12-15  
1  52.256860  42.771503          1                2021-07-15  
2  24.200033  26.473310          1                2021-02-03  
3  43.010880  44.695000          1                2020-02-07  




In [873]:
procedure_def(
"""
create or replace procedure ai_med.select_all_pacjenty()
language plpgsql    
as $$
declare
    liczba_pacjentow numeric;

    liczba_danych_o_lekach numeric;
    liczba_danych_o_morfologii numeric;
    liczba_danych_o_objawach numeric;
    liczba_danych_o_obrazach numeric;
    liczba_danych_o_operacjach numeric;
    liczba_danych_o_notatkach numeric;
    liczba_danych_o_leczeniu numeric;
    --srednia_liczba_danych_per_pacjent numeric;
begin

    select ai_med.test1() into liczba_pacjentow;

    select count(*) into liczba_danych_o_lekach from ai_med."Aktualnie przyjmowane leki";
    select count(*) into liczba_danych_o_morfologii from ai_med.morfologia;
    select count(*) into liczba_danych_o_objawach from ai_med.objawy;
    select count(*) into liczba_danych_o_obrazach from ai_med.obrazy;
    select count(*) into liczba_danych_o_operacjach from ai_med.operacje;
    select count(*) into liczba_danych_o_notatkach from ai_med."Nazwa pliku z notatkami lekarskimi";
    select count(*) into liczba_danych_o_leczeniu from ai_med."leczenie_w_kierunku_kod_ICD_10";

    RAISE NOTICE 'W bazie jest juz: % pacjentow!! Kazdy pacjent ma o sobie % infomacji.', liczba_pacjentow , ((liczba_danych_o_lekach + liczba_danych_o_morfologii + liczba_danych_o_objawach + liczba_danych_o_obrazach + liczba_danych_o_operacjach + liczba_danych_o_notatkach + liczba_danych_o_leczeniu)/liczba_pacjentow)/7;

end;$$
""",
commit=True)

#MOŻE JAKIŚ RETURN TU (PAMIETAJ ŻE MA BYĆ 1 WIERSZ) I TO JAKO JAKIŚ WIDO DLA LEKARZA? CHODZI MI ŻEBY DOSTĘP DO TEGO DLA JAKIEGOŚ LEKARZA 

In [874]:
do_query("""
call ai_med.select_all_pacjenty();
""", 
commit=True)

NOTICE:  W bazie jest juz: 50150 pacjentow!! Kazdy pacjent ma o sobie 0.99717134311351660000 infomacji.





In [875]:
#?JEŻELI PROCEDURA MOŻE PRZYJMOWAĆ JAKIEŚ ARGUMENTY? TO MOŻE PROCEDURA DO SPRAWDZANIA ZAKRESÓW NORM DLA JAKICHŚ PARAMETRÓW
#MOŻE TAKĄ FUNKCJE DODAĆ? 